# WeatherPy
----

In [ ]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
from citipy import citipy
from api_keys import api_key

# CSV with collected data
# output_data_file = 'WeatherData.csv'

# Set ranges of potential latitudes and longitudes
lat_set = (-90, 90)
long_set = (-180, 180)

## Generate Cities List

In [ ]:
# create empty lists for coordinates, cities/searchable city names, and country codes
lat_longs = []
cities = []
search_cities = [] # replaces each :space: in city name with :+: symbol allowing for usage in API web address
countries = []

# Create a set (min 500 :RESULTS:) of random coordinates
lats = np.random.uniform(low = -90.0000, high = 90.0000, size = 2000)
longs = np.random.uniform(low = -180.0000, high = 180.0000, size = 2000)
lat_longs = zip(lats, longs)

# Identify nearest city for each set of coordinates, add only if not already in list
for lat_long in lat_longs:
    city_name = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country_code = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    if city_name not in cities:
        cities.append(city_name)
        countries.append(country_code)

# Loop to clean cities for search_cities list. 
# Replaces the spaces in multi-word city names (so the address doesn't 'break' at space in API address)
for city in cities:
        readble_city = city.replace(' ', '+') # could alternately use '%20' instead of the '+', both work for this API!
        search_cities.append(readble_city)

# generate searchable item list for API
search_list = list(zip(search_cities, countries))

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Base API url for calls
url = 'http://api.openweathermap.org/data/2.5/weather?appid='
units = 'imperial'

In [ ]:
# # # # # # # # # # # # # # # # # #
#  API ADDRESS/JSON REVIEW BLOCK  #
# # # # # # # # # # # # # # # # # #

#  review data retrieved in call

# #  test variables - replace both of these with proper variables in iterable code
# city = 'Orlando'
# country = 'US'

# city_url = url + api_key + '&units=' + units + '&q=' + city + ',' + country
# response = requests.get(city_url) 
# weather_data = response.json() 

# #   test dumps for reading json retrieval
# print(json.dumps(weather_data, indent=3, sort_keys=True))

# #   test URL retrieval
# print(response.url)

In [ ]:
city_cap = [] # duplicates info from cities variable, but in a nicer format (proper capitalization)
country_cap = [] # duplicates info from countries variable, but in a nicer format (proper capitalization)
lat = []
temp = []
humidity = []
cloudiness = []
wind_speed = []
dates = []
failure_count = []
api_urls = []
call_response = []


print('''
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
=                                                 Processing Records                                                  =
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

''')

for call_count in range(len(search_list)):
    try:
        city_url = (url + api_key + '&units='+ units + '&q=' + search_list[call_count][0] + ',' + search_list[call_count][1])
        api_urls.append(city_url)
        weather_data = requests.get(city_url).json() 

        if weather_data["cod"] == '404':
            call_count += 1
            failure_count.append(f'City Call {call_count}')
            print(f'City Call {call_count}: ***** FAILED: City Not Found *****')
        elif weather_data["main"]["humidity"] >= 101:
            call_count += 1
            failure_count.append(f'City Call {call_count}')
            print(f'City Call {call_count}: {city_log}, {country_log} ***** DATA OUT OF BOUNDS: City Not Included *****')
        else:
            city_cap.append(weather_data["name"])
            city_log = weather_data["name"]
            country_cap.append(weather_data["sys"]["country"])
            country_log = weather_data["sys"]["country"]
            lat.append(weather_data["coord"]["lat"])
            temp.append(weather_data["main"]["temp_max"])
            humidity.append(weather_data["main"]["humidity"])
            cloudiness.append(weather_data["clouds"]["all"])
            wind_speed.append(weather_data["wind"]["speed"])
            timestamp = datetime.fromtimestamp(weather_data['dt']).strftime('%m/%d/%Y')
            dates.append(timestamp)

            call = (f'City Call {call_count + 1}:  {city_log}, {country_log}')
            call_response.append(call)
            print(call_response)

        call_count += 1

    except:
        print('= + = + = + = + = + = +     ERROR     + = + = + = + = + = + =')
    pass  


print('''
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
=                                              Data Collection Complete!                                              =
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
''')

# quick review/validation of lists 
print(f'''
{len(search_list)} random, unique cities generated.

{len(failure_count)} FAILURES TO CALL:
----------------------------------------------------------------------------------------------------------------------------
{failure_count}


Complete data will have {len(search_list) - len(failure_count)} items.

Item Count:
{len(city_cap)} Cities
{len(country_cap)} Countries
{len(lat)} Latitudes
{len(temp)} Temperatures
{len(humidity)} Humidity Percentages
{len(cloudiness)} Cloudiness Percentages
{len(wind_speed)} Wind Speeds
{len(dates)} Dates
''')

In [ ]:
output = open('WeatherData.csv', 'w')

with output as f:

    print(f'''
{len(search_list)} random, unique cities generated.

{len(failure_count)} FAILURES TO CALL: {failure_count}

Complete data will have {len(search_list) - len(failure_count)} items.

Item Count:
{len(city_cap)} Cities
{len(country_cap)} Countries
{len(lat)} Latitudes
{len(temp)} Temperatures
{len(humidity)} Humidity Percentages
{len(cloudiness)} Cloudiness Percentages
{len(wind_speed)} Wind Speeds
{len(dates)} Dates
''')

    print(f'Cities: {city_cap}')
    print(f'Country Codes: {country_cap}')
    print(f'Degrees Latitude: {lat}')
    print(f'Temperature: {temp}')
    print(f'Percent Humidity: {humidity}')
    print(f'Percent Cloudiness{cloudiness}')
    print(f'Wind Speed (mph): {wind_speed}')
    print(f'Date Reported: {dates}')
    print(f'Called Responses: {call_response}')
    print(f'Called URLs: {api_urls}')
    
output.close()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_df = (pd.DataFrame({'City': city_cap, 
                                 'Country': country_cap, 
                                 'Latitude': lat, 
                                 'Temperature (F)': temp, 
                                 'Humidity (%)': humidity, 
                                 'Cloudiness (%)': cloudiness, 
                                 'Wind Speed (mph)': wind_speed, 
                                 'Date Reported': dates}))
city_weather_df.head()

In [ ]:
# bad_data_df = city_weather_df[city_weather_df['Humidity (%)'] >= 101]

# bad_data_df.head(1000)

In [ ]:
# city_weather_df.drop(525, inplace = True)

In [ ]:
city_weather_df.count()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# variable for DATE in plots : represents date the information is called
called_on = datetime.now().strftime('%B %d, %Y')

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Temperature (F)'], 
            marker = 'o', 
            c = city_weather_df['Temperature (F)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Temperature\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Temperature: °Fahrenheit', fontsize = 16)
plt.ylim (20, 120)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()

plt.savefig('Temperature.png')
plt.show()

In [ ]:
# variable for DATE in plots : represents date the information is called
called_on = datetime.now().strftime('%B %d, %Y')

plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Temperature (F)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Temperature (F)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Temperatures by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Temperature: °Fahrenheit', fontsize = 16)
plt.xlim (20, 120)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (121, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (121, 80, 'North', fontsize = 13, color = 'navy')
plt.text (121, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig('Temperature_by_Latitude.png')

plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Humidity (%)'], 
            marker = 'o', 
            c = city_weather_df['Humidity (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Humidity\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Humidity', fontsize = 16)
plt.ylim (-2, 102)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig('Humidity.png')

plt.show()

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Humidity (%)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Humidity (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Humidity by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Percent Humidity', fontsize = 16)
plt.xlim (-2, 102)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (103, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (103, 80, 'North', fontsize = 13, color = 'navy')
plt.text (103, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig('Humidity_by_Latitude.png')

plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Cloudiness (%)'], 
            marker = 'o', 
            c = city_weather_df['Cloudiness (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Cloudiness\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Cloudiness', fontsize = 16)
plt.ylim (-2, 102)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig('Cloudiness.png')

plt.show()

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Cloudiness (%)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Cloudiness (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Cloudiness by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Percent Cloudiness', fontsize = 16)
plt.xlim (-2, 102)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (103, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (103, 80, 'North', fontsize = 13, color = 'navy')
plt.text (103, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig('Cloudiness_by_Latitude.png')

plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.figure(figsize = (13, 15))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Wind Speed (mph)'], 
            marker = 'o', 
            c = city_weather_df['Wind Speed (mph)'],            
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Wind Speed\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Wind Speed (mph)', fontsize = 16)
plt.ylim (-1, 31)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig('Wind_Speed.png')

plt.show()

In [ ]:
plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Wind Speed (mph)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Wind Speed (mph)'],            
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Wind Speed by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Wind Speed (mph)', fontsize = 16)
plt.xlim (-1, 31)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (31.5, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (31.5, 80, 'North', fontsize = 13, color = 'navy')
plt.text (31.5, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig('Wind_Speed_by_Latitude.png')

plt.show()

In [ ]:
## can I figure out how to adjust temp range to just 125 rather than the 150 below? can I manipulate enough to use? 

fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111, projection = 'polar')
ax.set_ybound(0,120)
c = ax.scatter(city_weather_df['Latitude'], 
               city_weather_df['Temperature (F)'], 
               c=city_weather_df['Temperature (F)'], 
               s = 300, 
               edgecolor = 'teal', 
               alpha = 0.75)


plt.show()